In [2]:
# Importing metadata from Culture Extractor to StashApp
# 1. Import metadata from Culture Extractor
# 2. Import metadata from StashApp by oshash
# 3. Join the two on oshash
# 4. Query metadata from StashDB by phash
# 5. Join the three on phash
# 6. Match performers between Culture Extractor, StashApp and StashDB
# 7. Set Culture Extractor UUIDs to performer custom fields in StashApp
# 8. Set metadata to StashApp scenes

In [3]:
import libraries.client_culture_extractor as client_culture_extractor
import os
import polars as pl
from dotenv import load_dotenv

load_dotenv()

# Culture Extractor
user = os.environ.get("CE_DB_USERNAME")
pw = os.environ.get("CE_DB_PASSWORD")
host = os.environ.get("CE_DB_HOST")
port = os.environ.get("CE_DB_PORT")
db = os.environ.get("CE_DB_NAME")

connection_string = f"dbname={db} user={user} password={pw} host={host} port={port}"

culture_extractor_client = client_culture_extractor.ClientCultureExtractor(connection_string)


# StashApp
from libraries.client_stashapp import StashAppClient, get_stashapp_client

stash_client = StashAppClient()
stash_raw_client = get_stashapp_client()


# StashDB
from libraries.StashDbClient import StashDbClient
import dotenv
import os

dotenv.load_dotenv()

stashbox_client = StashDbClient(
    os.getenv("STASHDB_ENDPOINT"),
    os.getenv("STASHDB_API_KEY"),
)


# Functions
def hex_to_binary(hex_string):
    return bin(int(hex_string, 16))[2:].zfill(64)

def calculate_hamming_distance(phash1, phash2):
    # Convert hexadecimal phashes to binary
    binary1 = hex_to_binary(phash1)
    binary2 = hex_to_binary(phash2)
    
    # Ensure both binary strings are of equal length
    if len(binary1) != len(binary2):
        raise ValueError("Binary strings must be of equal length")
    
    # Calculate Hamming distance
    return sum(c1 != c2 for c1, c2 in zip(binary1, binary2))

# Example usage:
# phash1 = "951428607cf7cb8f"
# phash2 = "951428607cf7cb8e"
# distance = calculate_hamming_distance(phash1, phash2)
# print(f"Hamming distance between {phash1} and {phash2}: {distance}")

def levenshtein(s1, s2):
    if not s1:
        return None
    if not s2:
        return None
    from Levenshtein import distance
    return distance(s1, s2)


dUsing stash (v0.27.2-37-g0621d871) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...
dUsing stash (v0.27.2-37-g0621d871) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...


In [4]:
all_tags = stash_raw_client.find_tags()

In [5]:
all_ce_sites = culture_extractor_client.get_sites()
all_ce_sites

ce_sites_uuid,ce_sites_short_name,ce_sites_name,ce_sites_url
str,str,str,str
"""018b94b1-b5e9-71d4-b67c-517b14…","""adultprime""","""Adult Prime""","""https://adultprime.com"""
"""018b94b1-b5e9-71d0-8ad9-c32b33…","""adulttime""","""Adult Time""","""https://members.adulttime.com/"""
"""018b94b1-b5e9-71c2-ab3d-b9e3b5…","""allfinegirls""","""AllFineGirls""","""https://venus.allfinegirls.com"""
"""018b94b1-b5e9-71c0-b340-3dfb48…","""alsscan""","""ALS Scan""","""https://www.alsscan.com"""
"""018dfdac-eb62-733a-a45c-d7d763…","""angelslove""","""angels.love""","""https://angels.love/members/"""
…,…,…,…
"""018b94b1-b5e9-71e0-a1e8-5fc6f6…","""wankzvr""","""WankzVR""","""https://www.wankzvr.com/"""
"""0190c3cb-5039-724d-84e5-abc7d6…","""watch4beauty""","""Watch4Beauty""","""https://www.watch4beauty.com"""
"""018b94b1-b5e9-71be-870e-b479d4…","""wowgirls""","""WowGirls""","""https://venus.wowgirls.com"""


In [6]:
all_ce_sub_sites = culture_extractor_client.get_sub_sites()
all_ce_sub_sites

ce_sites_uuid,ce_sites_short_name,ce_sites_name,ce_sites_url,ce_sub_sites_uuid,ce_sub_sites_short_name,ce_sub_sites_name
str,str,str,str,str,str,str
"""018b94b1-b5e9-71d4-b67c-517b14…","""adultprime""","""Adult Prime""","""https://adultprime.com""","""018b9407-b6be-7227-848f-febce9…","""4Kcfnm""","""4Kcfnm"""
"""018b94b1-b5e9-71d4-b67c-517b14…","""adultprime""","""Adult Prime""","""https://adultprime.com""","""018b9407-b6be-7228-8fab-a66fe1…","""Adultprime+Originals""","""Adultprime+Originals"""
"""018b94b1-b5e9-71d4-b67c-517b14…","""adultprime""","""Adult Prime""","""https://adultprime.com""","""0190f778-7307-7244-83e2-ab9151…","""Arousins""","""Arousins"""
"""018b94b1-b5e9-71d4-b67c-517b14…","""adultprime""","""Adult Prime""","""https://adultprime.com""","""018b9407-b6be-7229-b832-e65009…","""BBvideo""","""BBvideo"""
"""018b94b1-b5e9-71d4-b67c-517b14…","""adultprime""","""Adult Prime""","""https://adultprime.com""","""018b9407-b6be-722c-b044-68893e…","""BeautyAndTheSenior""","""BeautyAndTheSenior"""
…,…,…,…,…,…,…
"""018be1f2-ac2a-712a-94d9-9b7cef…","""tpdb""","""ThePornDB""","""https://api.metadataapi.net""","""018be25b-178d-7108-83cc-d646da…","""e77852ae-77c4-4449-b502-26075f…","""Slayed"""
"""018be1f2-ac2a-712a-94d9-9b7cef…","""tpdb""","""ThePornDB""","""https://api.metadataapi.net""","""018be241-76ee-75b4-b340-6743b0…","""4dfa7ae1-6b38-4226-89c9-c06755…","""Tushy"""
"""018be1f2-ac2a-712a-94d9-9b7cef…","""tpdb""","""ThePornDB""","""https://api.metadataapi.net""","""018be27c-f24b-7431-94b9-b59ac9…","""c9bb7f00-71fb-48f9-a69b-a59775…","""Tushy Raw"""


In [7]:
all_stash_studios = stash_client.get_studios()
all_stash_studios

stash_studios_id,stash_studios_name,stash_studios_url,stash_studios_stashdb_id,stash_studios_tpdb_id,stash_studios_ce_id,stash_studios_parent_studio_id,stash_studios_parent_studio_name,stash_studios_parent_studio_url,stash_studios_parent_studio_stashdb_id,stash_studios_parent_studio_tpdb_id,stash_studios_parent_studio_ce_id
i64,str,str,str,str,str,i64,str,str,str,str,str
404,"""1 Pass for All Sites""","""https://1passforallsites.com""","""441cbc73-d7be-43be-bd68-5fb26d…","""ebb49c58-e953-4f9b-97c1-bbfc68…",null,null,null,null,null,null,null
252,"""1000 Facials""","""https://www.1000facials.com/""","""5ee16943-0da6-4ee4-94c1-54172e…",null,null,234,"""Blowpass (Network)""","""https://www.blowpass.com""","""db258cb4-8a71-4468-9aa5-6ba0af…",null,null
569,"""1111 Customs""","""https://www.1111customs.com/""","""2fd594d3-ce39-4b85-8835-bc8bf9…",null,null,568,"""ChickPass Network""","""https://www.chickpassnetwork.c…","""2ab7ed11-7c93-4a41-8daf-881cc9…",null,null
1014,"""1111 Customs XXX""","""https://www.1111customs.com/""",null,"""40b4b673-fd8b-4f76-b11c-d01a41…",null,null,null,null,null,null,null
925,"""18 Eighteen""","""https://www.18eighteen.com/""","""ca22520a-e9b4-42a4-885b-6776e7…","""63127c59-e4e6-4cb5-b835-f62fb6…",null,1012,"""Porn Mega Load""","""https://pornmegaload.com""",null,"""5c62afb6-a310-4608-81bf-1eebe2…",null
…,…,…,…,…,…,…,…,…,…,…,…
1008,"""phil-flash""","""https://phil-flash.com/""","""c20c44c1-7c65-438c-825a-b21c80…",null,null,null,null,null,null,null,null
1214,"""pl3b""","""""",null,null,null,null,null,null,null,null,null
482,"""sensual.love""","""https://sensual.love/""","""1bd1caaf-5f7e-4d81-887c-ca9c2d…",null,null,221,"""Wow Network""","""""","""7ac565c8-3936-44a7-bd55-946460…",null,null


In [8]:
all_ce_sites_stash_studios_joined = all_ce_sites.join(all_stash_studios, left_on="ce_sites_name", right_on="stash_studios_name", how="left", coalesce=False)
all_ce_sites_stash_studios_joined

ce_sites_uuid,ce_sites_short_name,ce_sites_name,ce_sites_url,stash_studios_id,stash_studios_name,stash_studios_url,stash_studios_stashdb_id,stash_studios_tpdb_id,stash_studios_ce_id,stash_studios_parent_studio_id,stash_studios_parent_studio_name,stash_studios_parent_studio_url,stash_studios_parent_studio_stashdb_id,stash_studios_parent_studio_tpdb_id,stash_studios_parent_studio_ce_id
str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,str,str
"""018b94b1-b5e9-71d4-b67c-517b14…","""adultprime""","""Adult Prime""","""https://adultprime.com""",302,"""Adult Prime""","""https://adultprime.com/""","""bac1b01c-15af-430b-80af-1492a1…",null,null,null,null,null,null,null,null
"""018b94b1-b5e9-71d0-8ad9-c32b33…","""adulttime""","""Adult Time""","""https://members.adulttime.com/""",111,"""Adult Time""","""https://www.adulttime.com/""","""57381428-819a-452c-b647-31c41a…",null,null,502,"""Adult Time (Network)""","""https://www.adulttime.com/""","""6fa161d2-3c28-44b7-86a2-d98f5a…",null,null
"""018b94b1-b5e9-71c2-ab3d-b9e3b5…","""allfinegirls""","""AllFineGirls""","""https://venus.allfinegirls.com""",null,null,null,null,null,null,null,null,null,null,null,null
"""018b94b1-b5e9-71c0-b340-3dfb48…","""alsscan""","""ALS Scan""","""https://www.alsscan.com""",107,"""ALS Scan""","""https://www.alsscan.com/""","""df303e7f-09e5-4c29-a7db-ddd312…",null,null,305,"""MetArt Network""","""https://www.metartnetwork.com/""","""2c9d7ab5-3db2-4744-af76-d9e920…",null,null
"""018dfdac-eb62-733a-a45c-d7d763…","""angelslove""","""angels.love""","""https://angels.love/members/""",222,"""angels.love""","""https://angels.love/""","""c15b3aa6-3aaa-494a-abd8-8605b2…","""5ae98191-aee1-4ecc-9ffe-6446c0…",null,221,"""Wow Network""","""""","""7ac565c8-3936-44a7-bd55-946460…",null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""018b94b1-b5e9-71e0-a1e8-5fc6f6…","""wankzvr""","""WankzVR""","""https://www.wankzvr.com/""",249,"""WankzVR""","""https://www.wankzvr.com/""","""b04bca51-15ea-45ab-80f6-7b002f…","""16a2ce5b-3040-4e57-8651-3d10e9…",null,248,"""POVR Premium""","""https://www.povr.com""","""dcfd117f-a370-4402-b231-a9dd22…",null,null
"""0190c3cb-5039-724d-84e5-abc7d6…","""watch4beauty""","""Watch4Beauty""","""https://www.watch4beauty.com""",650,"""Watch4Beauty""","""https://www.watch4beauty.com/""","""bd3e299c-ce6d-4915-ad2d-4b5761…","""e559906f-808b-4bc1-be1f-8fd3dc…",null,null,null,null,null,null,null
"""018b94b1-b5e9-71be-870e-b479d4…","""wowgirls""","""WowGirls""","""https://venus.wowgirls.com""",2,"""WowGirls""","""https://www.wowgirls.com/""","""5d18fbd6-bc43-45ef-a146-c23c53…","""6c9144eb-cdf1-45cf-a84e-09a846…",null,221,"""Wow Network""","""""","""7ac565c8-3936-44a7-bd55-946460…",null,null


In [9]:
site_name = "Tushy"
all_ce_sites_stash_studios_joined.filter(pl.col("stash_studios_name").str.contains(site_name))

ce_sites_uuid,ce_sites_short_name,ce_sites_name,ce_sites_url,stash_studios_id,stash_studios_name,stash_studios_url,stash_studios_stashdb_id,stash_studios_tpdb_id,stash_studios_ce_id,stash_studios_parent_studio_id,stash_studios_parent_studio_name,stash_studios_parent_studio_url,stash_studios_parent_studio_stashdb_id,stash_studios_parent_studio_tpdb_id,stash_studios_parent_studio_ce_id
str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,str,str
"""018c057b-9336-71c0-90d9-8f47d2…","""tushy""","""Tushy""","""https://members.tushy.com""",115,"""Tushy""","""https://www.tushy.com/""","""eb58505f-428e-4a30-a151-bfab62…","""4dfa7ae1-6b38-4226-89c9-c06755…","""018c057b-9336-71c0-90d9-8f47d2…",212,"""Vixen Media Group""","""https://vixengroup.com""","""b62bc449-c3d9-49ff-9a16-8f5b1b…",null,null
"""018caef2-cb68-7633-b3e9-efe78a…","""tushyraw""","""Tushy Raw""","""https://members.tushyraw.com""",213,"""Tushy Raw""","""https://www.tushyraw.com/""","""be4be46f-692f-4509-ba23-90a96a…","""c9bb7f00-71fb-48f9-a69b-a59775…","""018caef2-cb68-7633-b3e9-efe78a…",212,"""Vixen Media Group""","""https://vixengroup.com""","""b62bc449-c3d9-49ff-9a16-8f5b1b…",null,null


In [10]:
rows = all_ce_sites_stash_studios_joined.filter(pl.col("stash_studios_name").str.contains(site_name))
selected_studio = rows.to_dicts()[0]
stash_client.set_studio_stash_id_for_endpoint(selected_studio["stash_studios_id"], "https://culture.extractor/graphql", selected_studio["ce_sites_uuid"])
selected_studio

{'ce_sites_uuid': '018c057b-9336-71c0-90d9-8f47d2e97d9d',
 'ce_sites_short_name': 'tushy',
 'ce_sites_name': 'Tushy',
 'ce_sites_url': 'https://members.tushy.com',
 'stash_studios_id': 115,
 'stash_studios_name': 'Tushy',
 'stash_studios_url': 'https://www.tushy.com/',
 'stash_studios_stashdb_id': 'eb58505f-428e-4a30-a151-bfab62b2694a',
 'stash_studios_tpdb_id': '4dfa7ae1-6b38-4226-89c9-c067557cc1c3',
 'stash_studios_ce_id': '018c057b-9336-71c0-90d9-8f47d2e97d9d',
 'stash_studios_parent_studio_id': 212,
 'stash_studios_parent_studio_name': 'Vixen Media Group',
 'stash_studios_parent_studio_url': 'https://vixengroup.com',
 'stash_studios_parent_studio_stashdb_id': 'b62bc449-c3d9-49ff-9a16-8f5b1bfa20b9',
 'stash_studios_parent_studio_tpdb_id': None,
 'stash_studios_parent_studio_ce_id': None}

In [11]:
downloads = culture_extractor_client.get_downloads(selected_studio["ce_sites_uuid"])
downloads

ce_downloads_site_name,ce_downloads_sub_site_name,ce_downloads_release_uuid,ce_downloads_release_date,ce_downloads_release_short_name,ce_downloads_release_name,ce_downloads_release_url,ce_downloads_release_description,ce_downloads_release_created,ce_downloads_release_last_updated,ce_downloads_release_available_files,ce_downloads_release_json_document,ce_downloads_uuid,ce_downloads_downloaded_at,ce_downloads_file_type,ce_downloads_content_type,ce_downloads_variant,ce_downloads_available_file,ce_downloads_original_filename,ce_downloads_saved_filename,ce_downloads_file_metadata,ce_downloads_performers,ce_downloads_tags,ce_downloads_hash_oshash,ce_downloads_hash_phash,ce_downloads_hash_sha256
str,str,str,date,str,str,str,str,datetime[μs],datetime[μs],str,str,str,datetime[μs],str,str,str,str,str,str,str,list[struct[4]],list[struct[4]],str,str,str
"""Tushy""",null,"""018c06d7-0363-76de-897c-6f561f…",2023-11-19,"""entanglements-part-2""","""Entanglements Part 2""","""https://members.tushy.com/vide…","""It was supposed to be four per…",2023-11-25 16:16:46.457003,2023-11-25 16:16:46.457062,"""[{""$type"":""AvailableVideoFile""…","""{""data"":{""findOneVideo"":{""id"":…","""018c0d0c-7f05-7604-98a1-880366…",2023-11-26 21:12:51.973189,"""image""","""poster""","""""","""{""$type"": ""AvailableImageFile""…","""poster.jpg""","""poster.webp""","""{""$type"": ""ImageFileMetadata"",…","[{""018c0678-f6be-73dd-840c-4a8b0398650d"",""alberto-blanco"",""Alberto Blanco"",""https://members.tushy.com/performers/alberto-blanco""}, {""018c0678-f6be-73dc-b76e-0e38ac8eada1"",""vince-karter"",""Vince Karter"",""https://members.tushy.com/performers/vince-karter""}, {""018c0678-f6be-73db-ad4a-040c4e8d60ce"",""jia-lissa"",""Jia Lissa"",""https://members.tushy.com/performers/jia-lissa""}]","[{""018c0678-f708-717b-be38-6a75e1c8bbe2"",""facial"",""facial"",""https://members.tushy.com/videos?search=facial""}, {""018c0678-f709-732d-b1eb-4fd73bc207cf"",""deep-throat"",""deep throat"",""https://members.tushy.com/videos?search=deep-throat""}, … {""018c0678-f709-733b-b93d-ea9e5982bd2f"",""double-blowjob"",""double blowjob"",""https://members.tushy.com/videos?search=double-blowjob""}]",null,null,"""c90eb3b00209fe8aeacbc9f4497f25…"
"""Tushy""",null,"""018c06d7-0363-76de-897c-6f561f…",2023-11-19,"""entanglements-part-2""","""Entanglements Part 2""","""https://members.tushy.com/vide…","""It was supposed to be four per…",2023-11-25 16:16:46.457003,2023-11-25 16:16:46.457062,"""[{""$type"":""AvailableVideoFile""…","""{""data"":{""findOneVideo"":{""id"":…","""018c0d0c-7b81-7132-9263-12b3b6…",2023-11-26 21:12:51.073687,"""image""","""carousel""","""8""","""{""$type"": ""AvailableImageFile""…","""carousel.jpg""","""carousel_8.jpg""","""{""$type"": ""ImageFileMetadata"",…","[{""018c0678-f6be-73dd-840c-4a8b0398650d"",""alberto-blanco"",""Alberto Blanco"",""https://members.tushy.com/performers/alberto-blanco""}, {""018c0678-f6be-73dc-b76e-0e38ac8eada1"",""vince-karter"",""Vince Karter"",""https://members.tushy.com/performers/vince-karter""}, {""018c0678-f6be-73db-ad4a-040c4e8d60ce"",""jia-lissa"",""Jia Lissa"",""https://members.tushy.com/performers/jia-lissa""}]","[{""018c0678-f708-717b-be38-6a75e1c8bbe2"",""facial"",""facial"",""https://members.tushy.com/videos?search=facial""}, {""018c0678-f709-732d-b1eb-4fd73bc207cf"",""deep-throat"",""deep throat"",""https://members.tushy.com/videos?search=deep-throat""}, … {""018c0678-f709-733b-b93d-ea9e5982bd2f"",""double-blowjob"",""double blowjob"",""https://members.tushy.com/videos?search=double-blowjob""}]",null,null,"""5e3a72c24cf25870cfa85d30352461…"
"""Tushy""",null,"""018c06d7-0363-76de-897c-6f561f…",2023-11-19,"""entanglements-part-2""","""Entanglements Part 2""","""https://members.tushy.com/vide…","""It was supposed to be four per…",2023-11-25 16:16:46.457003,2023-11-25 16:16:46.457062,"""[{""$type"":""AvailableVideoFile""…","""{""data"":{""findOneVideo"":{""id"":…","""018c0d0c-79a9-74e5-8d6a-701fd5…",2023-11-26 21:12:50.602008,"""

In [12]:
oshashes = downloads["ce_downloads_hash_oshash"].unique().to_list()
stash_app_scenes = stash_client.find_scenes_by_oshash(oshashes)
stash_app_scenes

stashapp_id,stashapp_title,stashapp_details,stashapp_date,stashapp_urls,stashapp_created_at,stashapp_updated_at,stashapp_performers,stashapp_studio,stashapp_files,stashapp_primary_file_path,stashapp_primary_file_basename,stashapp_primary_file_oshash,stashapp_primary_file_phash,stashapp_primary_file_xxhash,stashapp_primary_file_duration,stashapp_tags,stashapp_organized,stashapp_interactive,stashapp_play_duration,stashapp_play_count,stashapp_o_counter,stashapp_stash_ids,stashapp_stashdb_id,stashapp_tpdb_id,stashapp_ce_id
i64,str,str,date,list[str],datetime[μs],datetime[μs],list[struct[7]],struct[4],list[struct[6]],str,str,str,str,str,duration[ms],list[struct[2]],bool,bool,i64,i64,i64,list[struct[3]],str,str,str
2746,"""Back In Town""","""Sexy Lexi has an itch she need…",2020-02-20,"[""https://www.tushy.com/back-in-town"", ""https://tushy.com/videos/back-in-town"", … ""https://members.tushy.com/videos/back-in-town""]",2023-01-25 12:30:48,2025-01-06 12:23:36,"[{26,""Mick Blue"","""",[""Michael Austia"", ""Michael Austria"", … ""Miky Blue""],""MALE"",[{""https://stashdb.org/graphql"",""10b7b7d9-cc6f-4f90-ab78-cc082971400f"",1970-01-01 00:00:00}, {""https://theporndb.net/graphql"",""9ac1b398-106c-495b-887d-c588b89cba9d"",1970-01-01 00:00:00}],[{""CultureExtractor.nubilefilms"",""018ea23e-7ff2-71e3-92bf-26f86fdebdf6""}, {""CultureExtractor.tushy"",""018c0679-f40c-778d-9c30-d6b55778d015""}]}, {271,""Lexi Belle"","""",[""Lexi"", ""Lexi Bell"", … ""Nollie""],""FEMALE"",[{""https://stashdb.org/graphql"",""ed54d3f1-b19d-4776-8ab2-1237b17c7abb"",1970-01-01 00:00:00}, {""https://theporndb.net/graphql"",""f2a5f7d8-436c-4714-8a1e-71b809670451"",1970-01-01 00:00:00}, {""https://pmvstash.org/graphql"",""d4264757-c23f-48db-b78a-478deb29d9e7"",1970-01-01 00:00:00}],[{""CultureExtractor.tushy"",""018c0741-0f78-75fb-a941-9e0d88d5a980""}]}]","{115,""Tushy"",""https://www.tushy.com/"",{212,""Vixen Media Group"",""https://vixengroup.com""}}","[{2746,""W:\Culture\Videos\Sites\Vixen Media Group\Vixen Media Group꞉ Tushy\Vixen Media Group꞉ Tushy – 2020-02-20 – Back In Town – Lexi Belle, Mick Blue [b242fe7cc1e76405].mp4"",""Vixen Media Group꞉ Tushy – 2020-02-20 – Back In Town – Lexi Belle, Mick Blue [b242fe7cc1e76405].mp4"",7186069977,38m 4s 520ms,[{""oshash"",""648aa87bbee7620f""}, {""phash"",""b0b8cdeda34ccc07""}, … {""xxhash"",""b242fe7cc1e76405""}]}]","""W:\Culture\Videos\Sites\Vixen …","""Vixen Media Group꞉ Tushy – 202…","""648aa87bbee7620f""","""b0b8cdeda34ccc07""","""b242fe7cc1e76405""",38m 4s 520ms,"[{5179,""Ass to Mouth""}, {5313,""Blond Hair""}, … {7431,""Twosome (Straight)""}]",false,false,0,0,0,"[{""https://culture.extractor/graphql"",""018c0741-0455-77ec-bfbf-e516a539ea64"",2025-01-06 12:23:36.951124}, {""https://stashdb.org/graphql"",""de068ed2-2d45-47d8-8ba0-f1b8b0a40194"",2025-01-06 12:23:36.951124}, {""https://theporndb.net/graphql"",""46bfbea1-6643-450f-8f97-541ccf53f3a1"",1970-01-01 00:00:00}]","""de068ed2-2d45-47d8-8ba0-f1b8b0…","""46bfbea1-6643-450f-8f97-541ccf…","""018c0741-0455-77ec-bfbf-e516a5…"
2629,"""Hot Stress Relief""","""Alexis is training hard for a …",2019-12-02,"[""https://www.tushy.com/hot-stress-relief"", ""https://tushy.com/videos/hot-stress-relief"", … ""https://members.tushy.com/videos/hot-stress-relief""]",2023-01-25 12:29:13,2025-01-06 12:23:36,"[{139,""Alexis Crystal"","""",[""Anouk"", ""Carrie"", … ""Sindy""],""FEMALE"",[{""https://stashdb.org/graphql"",""d5061b46-796b-4204-8e4f-cff4569fdea6"",1970-01-01 00:00:00}, {""https://theporndb.net/graphql"",""1b5898d3-8ae1-4e36-9779-9961a77eb261"",1970-01-01 00:00:00}],[{""CultureExtractor.girlsonlyporn"",""018ea297-d4d3-77b1-b127-587bf49db84e""}, {""CultureExtractor.nubilefilms"",""018ea23e-da2f-7145-b066-cec3dd8fcd37""}, {""CultureExtractor.tushy"",""018c0741-dbab-7336-9238-d53d49ad6b3e""}]}, {396,""Christian Clay"","""",[""Chris Clay"", ""Christian"", … ""Sancio L'Orco""],""MALE"",[{""https://stashdb.org/graphql"",""26cdc70c-ba92-4028-bcab-d5a32802db19"",1970-01-01 00:0

In [13]:
joined_scenes = stash_app_scenes.join(downloads, left_on="stashapp_primary_file_oshash", right_on="ce_downloads_hash_oshash", how="left", coalesce=False)

In [14]:
# Create a list to store scene data
scene_data = []

# Create list of scene objects with filename, phash and duration
scene_objects = joined_scenes.select(
    pl.col("stashapp_primary_file_basename").alias("basename"),
    pl.col("stashapp_primary_file_phash").alias("phash"),
    pl.col("stashapp_primary_file_duration").dt.total_seconds().alias("duration"),
).to_dicts()

batch_size = 100

stashdb_scene_batches = []
for i in range(0, len(scene_objects), batch_size):
    batch = scene_objects[i:i+batch_size]
    batch_stashdb_scenes = stashbox_client.query_scenes_by_phash(batch)
    stashdb_scene_batches.append(batch_stashdb_scenes)

df_stashdb_scenes = pl.concat(stashdb_scene_batches)

In [15]:
joined_scenes = joined_scenes.join(df_stashdb_scenes, left_on="stashapp_primary_file_phash", right_on="queried_phash", how="left", coalesce=False)

In [19]:
# Get the stashapp_ids that have duplicates
duplicate_ids = (joined_scenes.group_by("stashapp_id")
                .agg(pl.col("stashapp_id").count().alias("scene_count"))
                .filter(pl.col("scene_count") > 1)
                .get_column("stashapp_id"))

# Show all rows for scenes that have duplicates
joined_scenes.filter(pl.col("stashapp_id").is_in(duplicate_ids)).sort("stashapp_id")

stashapp_id,stashapp_title,stashapp_details,stashapp_date,stashapp_urls,stashapp_created_at,stashapp_updated_at,stashapp_performers,stashapp_studio,stashapp_files,stashapp_primary_file_path,stashapp_primary_file_basename,stashapp_primary_file_oshash,stashapp_primary_file_phash,stashapp_primary_file_xxhash,stashapp_primary_file_duration,stashapp_tags,stashapp_organized,stashapp_interactive,stashapp_play_duration,stashapp_play_count,stashapp_o_counter,stashapp_stash_ids,stashapp_stashdb_id,stashapp_tpdb_id,stashapp_ce_id,ce_downloads_site_name,ce_downloads_sub_site_name,ce_downloads_release_uuid,ce_downloads_release_date,ce_downloads_release_short_name,ce_downloads_release_name,ce_downloads_release_url,ce_downloads_release_description,ce_downloads_release_created,ce_downloads_release_last_updated,ce_downloads_release_available_files,ce_downloads_release_json_document,ce_downloads_uuid,ce_downloads_downloaded_at,ce_downloads_file_type,ce_downloads_content_type,ce_downloads_variant,ce_downloads_available_file,ce_downloads_original_filename,ce_downloads_saved_filename,ce_downloads_file_metadata,ce_downloads_performers,ce_downloads_tags,ce_downloads_hash_oshash,ce_downloads_hash_phash,ce_downloads_hash_sha256,queried_phash,id,title,code,duration,date,urls,images,studio,tags,performers,fingerprints
i64,str,str,date,list[str],datetime[μs],datetime[μs],list[struct[7]],struct[4],list[struct[6]],str,str,str,str,str,duration[ms],list[struct[2]],bool,bool,i64,i64,i64,list[struct[3]],str,str,str,str,str,str,date,str,str,str,str,datetime[μs],datetime[μs],str,str,str,datetime[μs],str,str,str,str,str,str,str,list[struct[4]],list[struct[4]],str,str,str,str,str,str,str,duration[ms],date,list[struct[2]],list[struct[4]],struct[5],list[struct[2]],list[struct[2]],list[struct[3]]
15448,"""Vicarious Pleasure""","""Gianna knows Liya only invited…",2022-06-12,"[""https://www.tushy.com/videos/vicarious"", ""https://www.tushy.com/videos/vicarious-pleasure"", ""https://members.tushy.com/videos/vicarious-pleasure""]",2024-03-09 05:27:08,2025-01-06 12:23:39,"[{176,""Liya Silver"","""",[],""FEMALE"",[{""https://stashdb.org/graphql"",""88913713-a26d-4570-8b1d-59e0e2839185"",1970-01-01 00:00:00}, {""https://theporndb.net/graphql"",""39345e71-40ee-49ea-8dae-eaae5887d4e5"",1970-01-01 00:00:00}],[{""CultureExtractor.tushy"",""018c06db-7113-7054-8c35-9dafa25110c8""}]}, {396,""Christian Clay"","""",[""Chris Clay"", ""Christian"", … ""Sancio L'Orco""],""MALE"",[{""https://stashdb.org/graphql"",""26cdc70c-ba92-4028-bcab-d5a32802db19"",1970-01-01 00:00:00}, {""https://theporndb.net/graphql"",""0870fd45-0e74-4fd3-8319-b3c9613cda6a"",1970-01-01 00:00:00}],[{""CultureExtractor.tushy"",""018c06da-1a30-7150-8826-3ba2cac04379""}]}, {422,""Gianna Dior"","""",[],""FEMALE"",[{""https://stashdb.org/graphql"",""ceab303e-bc14-4acd-827b-7a6d95c179ae"",1970-01-01 00:00:00}, {""https://theporndb.net/graphql"",""97d453d5-2535-472e-9854-859d209f50b7"",1970-01-01 00:00:00}],[{""CultureExtractor.nubilefilms"",""018ea23e-da2f-7165-8b0e-a6b43b05a842""}, {""CultureExtractor.slayed"",""018c0789-9859-75e8-85b9-a2c48d7756cd""}, {""CultureExtractor.tushy"",""018c067a-0894-7513-927e-6ce2b01845fd""}]}]","{115,""Tushy"",""https://www.tushy.com/"",{212,""Vixen Media Group"",""https://vixengroup.com""}}","[{175098,""W:\Culture\Videos\Sites\Vixen Media Group\Vixen Media Group꞉ Tushy\Vixen Media Group꞉ Tushy – 2022-06-12 – Vicarious Pleasure – Gianna Dior, Liya Silver, Christian Clay [da6ae6b47b51b95b].mp4"",""Vixen Media Group꞉ Tushy – 2022-06-12 – Vicarious Pleasure – Gianna Dior, Liya Silver, Christian Clay [da6ae6b47b51b95b].mp4"",9781381818,51m 49s 660ms,[{""oshash"",""f1ac1cbc6f8ae205""}, {""phash"",""9f22233193345efa""}, … {""xxhash"",""da6ae6b47b51b95b""}]}]","""W:\Culture\Videos\Sites\Vixen …","""Vixen Media Group꞉ Tushy – 202…","""f1ac1cbc6f8ae205""","""9f22233193345efa""","""da6ae6b47b51b95b""",51m 49s 660ms,"[{5083,""All Anal""}, {5085,""All Natural""}, … {7405,""Trimmed Pussy""}]",false,f

In [15]:
# parquet_path = "joined_scenes_with_stashdb_scenes_20250105_1715.parquet"
# joined_scenes_with_stashdb_scenes.write_parquet(parquet_path)
# joined_scenes = pl.read_parquet(parquet_path)

In [16]:
def calculate_duration_difference(stashapp_duration, stashdb_duration):
    return (
        pl.when(stashapp_duration.is_not_null() & stashdb_duration.is_not_null())
        .then(
            ((stashapp_duration - stashdb_duration).abs() / 
             pl.max_horizontal([stashapp_duration, stashdb_duration])) * 100
        )
        .otherwise(None)
    )

def calculate_title_similarity(ce_title, stashdb_title):
    return (
        pl.when(ce_title.is_not_null() & stashdb_title.is_not_null())
        .then(
            pl.struct([ce_title, stashdb_title])
            .map_elements(
                lambda row: levenshtein(str(row[0]), str(row[1])),  # Access by index instead of field name
                return_dtype=pl.Int64
            )
        )
        .otherwise(None)
    )

def get_date_difference_days(ce_date, stashdb_date):
    return (
        pl.when(ce_date.is_not_null() & stashdb_date.is_not_null())
        .then(
            (ce_date.cast(pl.Datetime) - stashdb_date.cast(pl.Datetime)).dt.total_days().abs()
        )
        .otherwise(None)
    )

# First create the calculated columns
df_verification = joined_scenes.with_columns([
    calculate_duration_difference(
        pl.col("stashapp_primary_file_duration"), 
        pl.col("duration")
    ).alias("duration_diff_pct"),
    
    pl.struct(["ce_downloads_release_name", "title"])
        .map_elements(lambda x: levenshtein(x["ce_downloads_release_name"], x["title"]), return_dtype=pl.Int64)
        .alias("title_levenshtein"),
    
    get_date_difference_days(
        pl.col("ce_downloads_release_date"),
        pl.col("date")
    ).alias("date_diff_days"),
])

# Then add the warning flags
df_verification = df_verification.with_columns([
    # Add warning flags
    (pl.col("duration_diff_pct") > 5).alias("duration_warning"),
    (pl.col("title_levenshtein") > 5).alias("title_warning"),
    (pl.col("date_diff_days") > 7).alias("date_warning")
])

In [ ]:
joined_scenes_ce_unique_performers = (
    joined_scenes
    .select(pl.col("ce_downloads_performers"))
    .explode("ce_downloads_performers")
    .select([
        pl.col("ce_downloads_performers").struct.field("uuid").alias("performer_uuid"),
        pl.col("ce_downloads_performers").struct.field("name").alias("performer_name")
    ])
    .unique()
)
joined_scenes_ce_unique_performers

In [ ]:
all_stashapp_performers = stash_client.get_performers()
all_stashapp_performers = all_stashapp_performers.with_columns(
    pl.col("stashapp_custom_fields").list.eval(
        pl.when(pl.element().struct.field("key") == "CultureExtractor.tushy")
        .then(pl.element().struct.field("value"))
        .otherwise(None)
    ).list.eval(
        pl.element().filter(pl.element().is_not_null())
    ).list.first().alias("ce_custom_field_value")
)
all_stashapp_performers

In [ ]:
# Check for Culture Extractor performers that have not been matched to a StashApp performer
joined_scenes_ce_unique_performers.join(
    all_stashapp_performers.filter(pl.col("ce_custom_field_value").is_not_null()), 
    left_on="performer_uuid", 
    right_on="ce_custom_field_value", 
    how="left",  # Changed from "inner" to "left"
    coalesce=False
).filter(
    pl.col("ce_custom_field_value").is_null()  # Only show performers without matches
)

In [ ]:
from libraries.performer_matcher import PerformerMatcher

# Create matcher instance
matcher = PerformerMatcher(all_stashapp_performers)

# Your DataFrame already has the required columns, but we need to process each row
all_matches = []

# Process each row in your DataFrame
for row in joined_scenes.iter_rows(named=True):
    # Get performers from both sources
    ce_performers = row['ce_downloads_performers']
    stashapp_performers = row['stashapp_performers']
    
    # Create single-row DataFrame for the matcher
    scene_df = pl.DataFrame({
        'ce_downloads_performers': [ce_performers],
        'stashapp_performers': [stashapp_performers]
    })
    
    # Run matching for this scene
    matches = matcher.match_performers(
        scene_df['ce_downloads_performers'],
        scene_df['stashapp_performers']
    )
    
    # Add scene context to matches
    for match in matches:
        all_matches.append({
            'scene_id': row['stashapp_id'],
            'scene_title': row['stashapp_title'],
            'ce_uuid': match.ce_uuid,
            'ce_name': match.ce_name,
            'stashapp_id': match.stashapp_id,
            'stashapp_name': match.stashapp_name,
            'stashdb_uuid': match.stashdb_uuid,
            'stashdb_name': match.stashdb_name,
            'confidence': match.confidence,
            'reason': match.reason
        })

# Convert matches to DataFrame for analysis
matches_df = pl.DataFrame(all_matches)
matches_df

In [27]:
custom_field_name = "CultureExtractor." + selected_studio["ce_sites_short_name"]
for row in matches_df.select(pl.col(["ce_uuid", "stashapp_id"])).unique().iter_rows(named=True):
    stash_client.update_performer_custom_fields(row["stashapp_id"], {custom_field_name: row["ce_uuid"]})

In [ ]:
ce_performer_mapping = stash_client.get_performers().with_columns([
    pl.col("stashapp_custom_fields").list.eval(
        pl.element().struct.field("value").filter(
            pl.element().struct.field("key") == "CultureExtractor." + selected_studio["ce_sites_short_name"]
        )
    ).list.first().alias("ce_custom_field_value")
]).filter(
    pl.col("ce_custom_field_value").is_not_null()
).select(
    pl.col("ce_custom_field_value").alias("ce_performer_uuid"),
    pl.col("stashapp_id").alias("stashapp_id"),
    pl.col("stashapp_name").alias("stashapp_name")
).sort(by=["stashapp_name"])
ce_performer_mapping

In [ ]:
# First get all unique performer IDs from the scenes
unique_stashdb_performer_ids = joined_scenes.select([
    pl.col("performers").list.eval(
        pl.element().struct.field("performer").struct.field("id")
    )
]).explode(
    pl.col("performers")
).unique()
unique_stashdb_performer_ids

# Then join with StashApp performers that have StashDB IDs
stashdb_performer_mapping = stash_client.get_performers().with_columns([
    # Find the StashDB ID by filtering the stash_ids list first
    pl.col("stashapp_stash_ids").list.eval(
        pl.when(pl.element().struct.field("endpoint") == "https://stashdb.org/graphql")
        .then(pl.element().struct.field("stash_id"))
        .otherwise(None)
    ).list.eval(
        pl.element().filter(pl.element().is_not_null())
    ).list.first().alias("stashdb_id")
]).filter(
    pl.col("stashdb_id").is_not_null()
).select([
    pl.col("stashdb_id"),
    pl.col("stashapp_id"),
    pl.col("stashapp_name")
]).join(
    unique_stashdb_performer_ids,
    left_on="stashdb_id",
    right_on="performers",
    how="inner"
).sort(by=["stashapp_name"])
stashdb_performer_mapping

In [ ]:
# Get unique names from both mappings
ce_names = ce_performer_mapping.select("stashapp_name").unique()
stashdb_names = stashdb_performer_mapping.select("stashapp_name").unique()

# Find names in CE but not in StashDB (left difference)
names_only_in_ce = ce_names.join(
    stashdb_names, 
    on="stashapp_name", 
    how="anti"
)

# Find names in StashDB but not in CE (right difference)
names_only_in_stashdb = stashdb_names.join(
    ce_names, 
    on="stashapp_name", 
    how="anti"
)

if len(names_only_in_ce) > 0:
    print("Names only in Culture Extractor:")
    print(names_only_in_ce)
if len(names_only_in_stashdb) > 0:
    print("\nNames only in StashDB:")
    print(names_only_in_stashdb)

In [ ]:
downloads.filter(
    pl.col("ce_downloads_release_uuid").is_in(
        joined_scenes.get_column("ce_downloads_release_uuid").unique().to_list()
    )
).select(
    pl.col("ce_downloads_file_type"),
    pl.col("ce_downloads_content_type"),
    pl.col("ce_downloads_variant"),
).unique().sort(by=["ce_downloads_file_type", "ce_downloads_content_type", "ce_downloads_variant"])

In [ ]:
import base64

def create_update_dataframe(joined_scenes, downloads, all_stashapp_performers, all_tags, stashapp_studio_id):
    # Get all scene data ready for updates
    updates_df = joined_scenes.select([
        pl.col("ce_downloads_release_uuid").alias("ce_release_uuid"),
        pl.col("stashapp_id").alias("scene_id"),
        pl.col("stashapp_primary_file_basename").alias("primary_file_basename"),
        pl.col("ce_downloads_release_date").alias("date"),
        pl.col("ce_downloads_release_name").alias("title"),
        pl.col("ce_downloads_release_short_name").alias("code"),
        pl.col("ce_downloads_release_description").alias("details"),
        pl.lit(stashapp_studio_id).alias("studio_id"),
        pl.col("ce_downloads_release_url").alias("url"),
        pl.col("ce_downloads_release_uuid"),
        pl.col("id").alias("stashdb_id"),
        pl.col("ce_downloads_performers"),
        pl.col("performers"),
        pl.col("tags").alias("stashdb_tags")
    ])

    # Map performers - now with unique values
    updates_df = updates_df.with_columns([
        # Get Culture Extractor UUIDs
        pl.col("ce_downloads_performers").list.eval(
            pl.element().struct.field("uuid")
        ).list.unique().alias("ce_performer_uuids"),
        
        # Get StashDB IDs
        pl.col("performers").list.eval(
            pl.element().struct.field("performer").struct.field("id")
        ).list.unique().alias("stashdb_performer_ids")
    ])

    # Join performer IDs with unique values
    updates_df = updates_df.with_columns([
        pl.when(pl.col("ce_performer_uuids").is_not_null())
        .then(
            pl.col("ce_performer_uuids").map_elements(
                lambda uuids: ce_performer_mapping.filter(
                    pl.col("ce_performer_uuid").is_in(uuids)
                ).get_column("stashapp_id").unique().to_list(),
                return_dtype=pl.List(pl.Int64)
            )
        )
        .otherwise(pl.Series([[]]))
        .alias("ce_performer_stashapp_ids"),

        pl.when(pl.col("ce_performer_uuids").is_not_null())
        .then(
            pl.col("ce_performer_uuids").map_elements(
                lambda uuids: ce_performer_mapping.filter(
                    pl.col("ce_performer_uuid").is_in(uuids)
                ).get_column("stashapp_name").unique().to_list(),
                return_dtype=pl.List(pl.Utf8)
            )
        )
        .otherwise(pl.Series([[]]))
        .alias("ce_performer_stashapp_names")
    ])
    
    updates_df = updates_df.with_columns([
        pl.when(pl.col("performers").is_not_null())
        .then(
            pl.col("stashdb_performer_ids").map_elements(
                lambda uuids: stashdb_performer_mapping.filter(
                    pl.col("stashdb_id").is_in(uuids)
                ).get_column("stashapp_id").unique().to_list(),
                return_dtype=pl.List(pl.Int64)
            )
        )
        .otherwise(pl.Series([[]]))
        .alias("stashdb_performer_stashapp_ids"),

        pl.when(pl.col("stashdb_performer_ids").is_not_null())
        .then(
            pl.col("stashdb_performer_ids").map_elements(
                lambda uuids: stashdb_performer_mapping.filter(
                    pl.col("stashdb_id").is_in(uuids)
                ).get_column("stashapp_name").unique().to_list(),
                return_dtype=pl.List(pl.Utf8)
            )
        )
        .otherwise(pl.Series([[]]))
        .alias("stashdb_performer_stashapp_names")
    ])

    # Combine performer IDs with unique values
    updates_df = updates_df.with_columns([
        pl.concat_list([
            pl.col("ce_performer_stashapp_ids"),
            pl.col("stashdb_performer_stashapp_ids")
        ]).list.unique().alias("performer_ids"),
        
        pl.concat_list([
            pl.col("ce_performer_stashapp_names"),
            pl.col("stashdb_performer_stashapp_names")
        ]).list.unique().alias("performer_names")
    ])

    # Map tags
    tag_mapping = pl.DataFrame({
        "stashdb_name": [tag["name"] for tag in all_tags],
        "stashapp_id": [tag["id"] for tag in all_tags]
    })

    updates_df = updates_df.with_columns([
        pl.when(pl.col("stashdb_tags").is_not_null())
        .then(
            pl.col("stashdb_tags").map_elements(
                lambda tags: tag_mapping.filter(
                    pl.col("stashdb_name").is_in([t["name"] for t in tags])
                ).get_column("stashapp_id").to_list(),
                return_dtype=pl.List(pl.Utf8)
            )
        )
        .otherwise(pl.Series([[]]))
        .alias("tag_ids")
    ])

    # Get scene images
    scene_images = downloads.filter(
        pl.col("ce_downloads_file_type") == "image",
        pl.col("ce_downloads_content_type").is_in(["poster", "scene"])
    ).select([
        pl.col("ce_downloads_release_uuid"),
        pl.col("ce_downloads_saved_filename").alias("scene_image_filename")
    ])

    # Get gallery info
    galleries = downloads.filter(
        (pl.col("ce_downloads_content_type") == "gallery") &
        (pl.col("ce_downloads_variant").is_in(["Large", ""]))
    ).select([
        pl.col("ce_downloads_release_uuid"),
        pl.col("ce_downloads_hash_sha256").alias("gallery_hash")
    ])

    # Join images and galleries
    updates_df = updates_df.join(
        scene_images,
        on="ce_downloads_release_uuid",
        how="left"
    ).join(
        galleries,
        on="ce_downloads_release_uuid",
        how="left"
    )

    return updates_df

def generate_update_inputs(updates_df, stash_raw_client):
    updates = []
    
    for row in updates_df.iter_rows(named=True):
        # Get current scene data
        refreshed_scene = stash_raw_client.find_scene(row["scene_id"])
        
        # Load scene image
        try:
            image_path = os.path.join(
                "F:\\Ripping\\" + selected_studio["ce_sites_name"] + "\\Metadata", 
                row["ce_downloads_release_uuid"],
                row["scene_image_filename"]
            )
            scene_image_base64 = base64.b64encode(open(image_path, "rb").read()).decode("utf-8")
        except Exception as e:
            scene_image_base64 = None

        # Find gallery if exists
        gallery_id = None
        refreshed_gallery = None  # Initialize refreshed_gallery
        existing_gallery_urls = []
        if row["gallery_hash"]:
            found_galleries = stash_raw_client.find_galleries(q=row["gallery_hash"])
            if len(found_galleries) == 1:
                gallery_id = found_galleries[0]["id"]
                refreshed_gallery = stash_raw_client.find_gallery(gallery_id)
                existing_gallery_urls = refreshed_gallery.get("urls", [])

        # Handle potentially null values
        existing_scene_tag_ids = sorted([tag["id"] for tag in refreshed_scene.get("tags", [])])
        existing_gallery_tag_ids = sorted([tag["id"] for tag in refreshed_gallery.get("tags", [])]) if refreshed_gallery else []
        new_tag_ids = sorted(row["tag_ids"]) if row["tag_ids"] is not None else []
        
        existing_performer_ids = [int(performer["id"]) for performer in refreshed_scene.get("performers", [])]
        new_performer_ids = row["performer_ids"] if row["performer_ids"] is not None else []
        
        existing_urls = refreshed_scene.get("urls", [])
        new_url = [row["url"]] if row["url"] is not None else []
        
        existing_stash_ids = refreshed_scene.get("stash_ids", [])

        new_stash_ids = []
        if row.get("stashdb_id"):
            new_stash_ids.append({
                "endpoint": "https://stashdb.org/graphql",
                "stash_id": row["stashdb_id"]
            })
        if row.get("ce_downloads_release_uuid"):
            new_stash_ids.append({
                "endpoint": "https://culture.extractor/graphql", 
                "stash_id": row["ce_downloads_release_uuid"]
            })

        scene_stash_ids = list({
            (stash_id["endpoint"], stash_id["stash_id"]): stash_id
            for stash_id in existing_stash_ids + new_stash_ids
        }.values())

        update = {
            "ce_release_uuid": row["ce_release_uuid"],
            "scene_id": row["scene_id"],
            "primary_file_basename": row["primary_file_basename"],
            "existing_scene_gallery_id": refreshed_scene.get("gallery_id", None),
            "gallery_id": gallery_id,
            "existing_scene_date": refreshed_scene.get("date", None),
            "existing_gallery_date": refreshed_gallery.get("date", None) if refreshed_gallery else None,
            "date": row["date"].strftime("%Y-%m-%d") if row["date"] else None,
            "existing_scene_title": refreshed_scene.get("title", None),
            "existing_gallery_title": refreshed_gallery.get("title", None) if refreshed_gallery else None,
            "title": row["title"],
            "existing_scene_code": refreshed_scene.get("code", None),
            "existing_gallery_code": refreshed_gallery.get("code", None) if refreshed_gallery else None,
            "code": row["code"],
            "existing_scene_details": refreshed_scene.get("details", None),
            "existing_gallery_details": refreshed_gallery.get("details", None) if refreshed_gallery else None,
            "details": row["details"],
            "existing_scene_studio_id": refreshed_scene.get("studio", None).get("id", None),
            "existing_gallery_studio_id": refreshed_gallery.get("studio", None).get("id", None) if refreshed_gallery else None,
            "studio_id": row["studio_id"],
            "existing_scene_performers": refreshed_scene.get("performers", []),
            "existing_gallery_performers": refreshed_gallery.get("performers", []) if refreshed_gallery else [],
            "performer_ids": list(set(existing_performer_ids + new_performer_ids)),
            "existing_scene_tags": existing_scene_tag_ids,
            "existing_gallery_tags": existing_gallery_tag_ids,
            "scene_tag_ids": sorted(list(set(existing_scene_tag_ids + new_tag_ids))),
            "gallery_tag_ids": sorted(list(set(existing_gallery_tag_ids + new_tag_ids))) if refreshed_gallery else [],
            "existing_scene_urls": refreshed_scene.get("urls", []),
            "scene_urls": existing_urls + new_url,
            "existing_gallery_urls": refreshed_gallery.get("urls", []) if refreshed_gallery else [],
            "gallery_urls": (existing_gallery_urls + [
                row["url"],
                f"https://culture.extractor/galleries/{row['ce_downloads_release_uuid']}"
            ]) if gallery_id else None,
            "cover_image": f"data:image/jpeg;base64,{scene_image_base64}",
            "scene_stash_ids": scene_stash_ids
        }
        updates.append(update)

    return pl.DataFrame(updates)

# Usage
updates_df = create_update_dataframe(
    joined_scenes,
    downloads,
    all_stashapp_performers,
    all_tags,
    selected_studio["stash_studios_id"]
)

update_inputs_df = generate_update_inputs(updates_df, stash_raw_client)

update_inputs_df = update_inputs_df.sort(by=["date"])

# Review updates before applying
print("Updates to be applied:")
print(update_inputs_df)


In [ ]:
# Apply updates if everything looks good
for update in update_inputs_df.iter_rows(named=True):
    # Update scene
    scene_input = {
        "id": update["scene_id"],
        "date": update["date"],
        "title": update["title"],
        "code": update["code"],
        "details": update["details"],
        "studio_id": update["studio_id"],
        "performer_ids": update["performer_ids"],
        "tag_ids": update["scene_tag_ids"],
        "urls": update["scene_urls"],
        "cover_image": update["cover_image"],
        "stash_ids": update["scene_stash_ids"]
    }
    if update["gallery_id"]:
        scene_input["gallery_ids"] = [update["gallery_id"]]
    
    try:
        stash_raw_client.update_scene(scene_input)
    except Exception as e:
        print(f"Error updating scene for {update['scene_id']}: {e}")
        continue

    # Update gallery if exists
    if update["gallery_id"]:
        gallery_input = {
            "id": update["gallery_id"],
            "date": update["date"],
            "title": update["title"],
            "code": update["code"],
            "details": update["details"],
            "studio_id": update["studio_id"],
            "performer_ids": update["performer_ids"],
            "tag_ids": update["gallery_tag_ids"],
            "urls": update["gallery_urls"]
        }
        try:
            stash_raw_client.update_gallery(gallery_input)
        except Exception as e:
            print(f"Error updating gallery for {update['scene_id']}: {e}")
            continue
